## 11. Train the model for a Q&A task that takes a context as addi'onal input along with the queston. You can use SQuAD dataset (ttps://rajpurkar.github.io/SQuAD-explorer/ ) or the smaller Topioca dataset (https://## mcgill-nlp.github.io/topiocqa/) . Choose an appropriate task prefix/trigger word and jus'fy the choice.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

In [2]:
#from datasets import load_dataset
#dataset = load_dataset("squad", data_files="/Users/prabhatkumar/Downloads/train-v2.0.json")

In [3]:
import pandas as pd
def apply_dataframe(row):
    df = pd.DataFrame(row['paragraphs'])
    df['title'] = row['title']
    return df

def apply_dataframe1(row):
    #print(row['qas'])
    if 'plausible_answers' in row['qas']:
        row['qas']['answers'] = row['qas']['plausible_answers']
        del row['qas']['plausible_answers']
    df = pd.DataFrame(row['qas'])
    df['title'] = row['title']
    df['context'] = row['context']
    
    return df
        

train = pd.read_json("/Users/prabhatkumar/Downloads/train-v2.0.json")
valid = pd.read_json("/Users/prabhatkumar/Downloads/dev-v2.0.json")

In [4]:
def create_df_from_json(df):
    df['text1'] = df['data'].apply(pd.DataFrame)
    df = pd.concat(df['text1'].tolist())
    df = df.reset_index(drop=True)
    df['text1'] = df.apply(apply_dataframe,axis=1)
    df = pd.concat(df['text1'].tolist())
    df['text1'] = df.apply(apply_dataframe1,axis=1)
    df['text2'] = df['text1'].apply(lambda x : len(x.columns.tolist()))
    df = pd.concat(df['text1'].tolist())
    df['answer'] = df['answers'].apply(lambda x: x['text'])

    df=df[['id','title','question','context','answer']]
    df = df.drop_duplicates()
    df=df.reset_index(drop=True)
    return df

In [5]:
train =create_df_from_json(train)
valid =create_df_from_json(valid)

In [6]:
train.head()

,id,title,question,context,answer
0,56be85543aeaaa14008c9063,Beyoncé,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s
1,56be85543aeaaa14008c9065,Beyoncé,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing
2,56be85543aeaaa14008c9066,Beyoncé,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003
3,56bf6b0f3aeaaa14008c9601,Beyoncé,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,Beyoncé,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s


In [7]:
valid.head()

,id,title,question,context,answer
0,56ddde6b9a695914005b9628,Normans,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,France
1,56ddde6b9a695914005b9629,Normans,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,10th and 11th centuries
2,56ddde6b9a695914005b9629,Normans,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,in the 10th and 11th centuries
3,56ddde6b9a695914005b962a,Normans,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,"Denmark, Iceland and Norway"
4,56ddde6b9a695914005b962b,Normans,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,Rollo


In [17]:
train = train.loc[0:3500]
valid = valid.loc[0:1000]

In [18]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train)
valid_dataset = Dataset.from_pandas(valid)



#### selected question: question context: context because the model T5 FLAN is already trained on this pattern for question answer so providing this template/prompt as examples for training. We could have tried other template if we are including domain question awnser. ALso model has been fully trained for all layers instead of finetuning a few layers because the data is huge. The ideal way is to train all layers for few epochs then freeze all layers expect the head or few layers of decoder and encoder.

#### In this notebook I had to use only sample of data to train since colab was going timeout and I dont have a GPU powered laptop

In [19]:
def preprocess_function(examples):
    inputs = [f"question: {q} context: {c}" for q, c in zip(examples["question"], examples["context"])]
    targets = examples["answer"]
    #print(inputs)
    #print(targets)
    #inputs = tokenizer(inputs, padding=True, truncation=True,return_tensors="pt")
    #targets = tokenizer(targets, padding=True, truncation=True,return_tensors="pt")
    inputs = tokenizer(inputs, padding='max_length', truncation=True, max_length=512,return_tensors="pt")
    targets = tokenizer(targets, padding='max_length', truncation=True, max_length=512,return_tensors="pt")
    
    return {
        'input_ids': inputs.input_ids,
        'attention_mask': inputs.attention_mask,
        'decoder_input_ids': targets.input_ids,
        'labels': targets.input_ids,
    }

    #return {"input_ids": tokenizer(inputs, padding="max_length", max_length=64, truncation=True), "labels": tokenizer(targets, padding="max_length", max_length=32, truncation=True)}



In [20]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3501 [00:00<?, ? examples/s]

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [21]:
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from dataclasses import dataclass
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union
from transformers.utils import PaddingStrategy
import numpy as np
@dataclass
class CustomDataCollatorForSeq2Seq:
    """
    Data collator that will dynamically pad the inputs received, as well as the labels.

    Args:
        tokenizer ([`PreTrainedTokenizer`] or [`PreTrainedTokenizerFast`]):
            The tokenizer used for encoding the data.
        model ([`PreTrainedModel`]):
            The model that is being trained. If set and has the *prepare_decoder_input_ids_from_labels*, use it to
            prepare the *decoder_input_ids*

            This is useful when using *label_smoothing* to avoid calculating loss twice.
        padding (`bool`, `str` or [`~utils.PaddingStrategy`], *optional*, defaults to `True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:

            - `True` or `'longest'` (default): Pad to the longest sequence in the batch (or no padding if only a single
              sequence is provided).
            - `'max_length'`: Pad to a maximum length specified with the argument `max_length` or to the maximum
              acceptable input length for the model if that argument is not provided.
            - `False` or `'do_not_pad'`: No padding (i.e., can output a batch with sequences of different lengths).
        max_length (`int`, *optional*):
            Maximum length of the returned list and optionally padding length (see above).
        pad_to_multiple_of (`int`, *optional*):
            If set will pad the sequence to a multiple of the provided value.

            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
        label_pad_token_id (`int`, *optional*, defaults to -100):
            The id to use when padding the labels (-100 will be automatically ignored by PyTorch loss functions).
        return_tensors (`str`):
            The type of Tensor to return. Allowable values are "np", "pt" and "tf".
    """

    tokenizer: PreTrainedTokenizerBase
    model: Optional[Any] = None
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    label_pad_token_id: int = -100
    return_tensors: str = "pt"

    def __call__(self, features, return_tensors=None):
        if return_tensors is None:
            return_tensors = self.return_tensors
        labels = [feature["labels"] for feature in features] if "labels" in features[0].keys() else None
        
        # We have to pad the labels before calling `tokenizer.pad` as this method won't pad them and needs them of the
        # same length to return tensors.
        if labels is not None:
            max_label_length = max(len(l) for l in labels)
            if self.pad_to_multiple_of is not None:
                max_label_length = (
                    (max_label_length + self.pad_to_multiple_of - 1)
                    // self.pad_to_multiple_of
                    * self.pad_to_multiple_of
                )

            padding_side = self.tokenizer.padding_side
            for feature in features:
                remainder = [self.label_pad_token_id] * (max_label_length - len(feature["labels"]))
                if isinstance(feature["labels"], list):
                    feature["labels"] = (
                        feature["labels"] + remainder if padding_side == "right" else remainder + feature["labels"]
                    )
                elif padding_side == "right":
                    feature["labels"] = np.concatenate([feature["labels"], remainder]).astype(np.int64)
                else:
                    feature["labels"] = np.concatenate([remainder, feature["labels"]]).astype(np.int64)

        features = self.tokenizer.pad(
            features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=return_tensors,
        )
        print(features)
        # prepare decoder_input_ids
        if (
            labels is not None
            and self.model is not None
            and hasattr(self.model, "prepare_decoder_input_ids_from_labels")
        ):
            decoder_input_ids = self.model.prepare_decoder_input_ids_from_labels(labels=features["labels"])
            features["decoder_input_ids"] = decoder_input_ids

        return features


data_collator = DataCollatorForSeq2Seq(model=model,tokenizer=tokenizer)

In [22]:
train.head()

,id,title,question,context,answer
0,56be85543aeaaa14008c9063,Beyoncé,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s
1,56be85543aeaaa14008c9065,Beyoncé,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing
2,56be85543aeaaa14008c9066,Beyoncé,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003
3,56bf6b0f3aeaaa14008c9601,Beyoncé,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,Beyoncé,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s


In [23]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./t5-question-answering",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_steps=10000,
    evaluation_strategy="steps",
    eval_steps=2000,
    save_total_limit=2,
    load_best_model_at_end=True,
    push_to_hub=False,
    remove_unused_columns=True,
)

In [24]:

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

In [25]:
trainer.train()


/opt/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/657 [00:00<?, ?it/s]

wandb: Network error (ConnectTimeout), entering retry loop.


{'loss': 2.2356, 'learning_rate': 1.1948249619482495e-05, 'epoch': 2.28}
{'train_runtime': 23722.254, 'train_samples_per_second': 0.443, 'train_steps_per_second': 0.028, 'train_loss': 1.7246996169765245, 'epoch': 3.0}


TrainOutput(global_step=657, training_loss=1.7246996169765245, metrics={'train_runtime': 23722.254, 'train_samples_per_second': 0.443, 'train_steps_per_second': 0.028, 'train_loss': 1.7246996169765245, 'epoch': 3.0})

In [26]:
results = trainer.evaluate()


  0%|          | 0/63 [00:00<?, ?it/s]

In [27]:
results

{'eval_loss': 0.027084223926067352,
 'eval_runtime': 484.0654,
 'eval_samples_per_second': 2.068,
 'eval_steps_per_second': 0.13,
 'epoch': 3.0}

In [28]:
trainer.save_model("./t5-question-answering-finetuned")


In [29]:
model_new = T5ForConditionalGeneration.from_pretrained("./t5-question-answering-finetuned") 

In [30]:
pipe = pipeline("text2text-generation", model=model_new,tokenizer=tokenizer)
text="question: What is the age of Greg chappel? context: Cricket legend Greg Chappell has revealed going through a financial struggle with his friends pitching in to set up an online fundraising platform to enhance his last few years, according to a report. The 75-year-old former Australia captain, who also had a controversial stint as the head coach of the Indian team from 2005-2007, admitted that he is doing fine but is certainly not living a life of luxury as a result of his cricketing career."
pipe(text)

[{'generated_text': '75-year-old'}]

## 12. Evaluate the quality of the model

In [32]:
eval = pd.read_json("/Users/prabhatkumar/Downloads/dev-v2.0.json")
eval =create_df_from_json(eval)

In [33]:
eval = eval[2000:3000]

In [34]:
eval.head()

,id,title,question,context,answer
2000,5ad26c87d7d075001a42939c,Huguenot,Which English town had the largest refugee chu...,Other evidence of the Walloons and Huguenots i...,Canterbury
2001,57109180a58dae1900cd6a40,Huguenot,What Irish cities had Huguenot mayors in the 1...,A number of Huguenots served as mayors in Dubl...,"Dublin, Cork, Youghal and Waterford"
2002,57109180a58dae1900cd6a41,Huguenot,French Church Street is in what Irish town?,A number of Huguenots served as mayors in Dubl...,Cork City
2003,57109180a58dae1900cd6a42,Huguenot,Where is D'Olier Street?,A number of Huguenots served as mayors in Dubl...,Dublin
2004,57109180a58dae1900cd6a43,Huguenot,D'Olier Street is named after whom?,A number of Huguenots served as mayors in Dubl...,a High Sheriff and one of the founders of the ...


In [35]:
questions = eval['question'].tolist()
context = eval['context'].tolist()
actual_answer = eval['answer'].tolist()

In [36]:
pred_answers=[]
for q,c in zip(questions,context):
    text=f"question: {q} context: {c}"
    pred_answers.append(pipe(text))

In [41]:
pred_answer_mod = [col[0]['generated_text'] for col  in pred_answers]

In [44]:
import difflib

def answer_evaluation(actual_answer, predicted_answer,threshold=0.9):
    actual_answer = actual_answer.lower()  
    predicted_answer = predicted_answer.lower()

    similarity = difflib.SequenceMatcher(None, actual_answer, predicted_answer).ratio()

    if similarity >= threshold:
        return "Exact Match"
    elif similarity >= 0.7:
        return "High Partial Match"
    elif similarity >= 0.4:
        return "Low Partial Match"
    else:
        return "No Match"

In [49]:
response_list = []
for act,pred in zip(actual_answer,pred_answer_mod):
    #print(act,'....',pred)
    response_list.append(answer_evaluation(act, pred,threshold=0.9))

In [56]:
eval_df = pd.DataFrame()
eval_df['questions'] = questions
eval_df['context'] = context
eval_df['actual_answer'] = actual_answer
eval_df['predicted_answer'] = pred_answer_mod
eval_df['evaluation_criteria'] = response_list

In [57]:
eval_df.head()

,questions,context,actual_answer,predicted_answer,evaluation_criteria
0,Which English town had the largest refugee chu...,Other evidence of the Walloons and Huguenots i...,Canterbury,"Sandwich, Faversham and Maidstone",No Match
1,What Irish cities had Huguenot mayors in the 1...,A number of Huguenots served as mayors in Dubl...,"Dublin, Cork, Youghal and Waterford","Dublin, Cork, Youghal and Waterford",Exact Match
2,French Church Street is in what Irish town?,A number of Huguenots served as mayors in Dubl...,Cork City,Portarlington,No Match
3,Where is D'Olier Street?,A number of Huguenots served as mayors in Dubl...,Dublin,Dublin,Exact Match
4,D'Olier Street is named after whom?,A number of Huguenots served as mayors in Dubl...,a High Sheriff and one of the founders of the ...,High Sheriff,No Match


#### below is evaluation based on No Match, Exact Match and partial match. This type of evaluation can be tweeked to take business requirement and criteria into consideration. We can implement further logic on the outcome and create a formula that will give us one score. To be honest this logic that I used can be further improved as per the business criteria

In [64]:
eval_df['evaluation_criteria'].value_counts()

No Match              522
Exact Match           254
Low Partial Match     151
High Partial Match     73
Name: evaluation_criteria, dtype: int64

In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def get_metrics(actual_answer, predicted_answers):
    accuracy = accuracy_score(actual_answer, predicted_answers)
    precision = precision_score(actual_answer, predicted_answers, average='macro')
    recall = recall_score(actual_answer, predicted_answers, average='macro')
    f1 = f1_score(actual_answer, predicted_answers, average='macro')

    results = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
    }

    return results

In [54]:
metrics = get_metrics(actual_answer,pred_answer_mod)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### below is evaluation based on metrices

In [55]:
metrics

{'Accuracy': 0.217,
 'Precision': 0.11146521501682433,
 'Recall': 0.1423510913799482,
 'F1 Score': 0.11523753715762595}